In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

def eda_template(df):
    # Data Overview
    print("🔍 Data Overview")
    print(df.head())
    print("\n📏 Shape of Data:", df.shape)
    print("\n🧾 Columns:", df.columns.tolist())

    # Missing Values
    print("\n📉 Missing Values")
    print(df.isnull().sum())

    # Data Types
    print("\n🔢 Data Types")
    print(df.dtypes)

    # Summary Statistics
    print("\n📊 Summary Statistics")
    print(df.describe(include='all'))

    # Detect datetime columns
    datetime_cols = df.select_dtypes(include=['datetime64', 'datetime']).columns.tolist()
    if not datetime_cols:
        for col in df.columns:
            try:
                df[col] = pd.to_datetime(df[col])
                datetime_cols.append(col)
                break
            except:
                continue

    # Univariate Analysis - Numerical
    print("\n📈 Univariate Analysis - Numerical Features")
    num_cols = df.select_dtypes(include=np.number).columns
    for col in num_cols:
        plt.figure(figsize=(6, 4))
        sns.histplot(df[col].dropna(), kde=True)
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.show()

    # Univariate Analysis - Categorical
    print("\n📊 Univariate Analysis - Categorical Features")
    cat_cols = df.select_dtypes(include='object').columns
    for col in cat_cols:
        plt.figure(figsize=(6, 4))
        df[col].value_counts().plot(kind='bar')
        plt.title(f'Value Counts of {col}')
        plt.xlabel(col)
        plt.ylabel('Count')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    # Bivariate Analysis - Numerical vs Numerical
    print("\n📉 Bivariate Analysis - Numerical vs Numerical")
    if len(num_cols) >= 2:
        sns.pairplot(df[num_cols].dropna())
        plt.suptitle('Pairplot of Numerical Features', y=1.02)
        plt.show()

    # Correlation Analysis
    print("\n🔗 Correlation Matrix")
    corr = df[num_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Heatmap')
    plt.tight_layout()
    plt.show()

    # Outlier Detection
    print("\n🚨 Outlier Detection")
    for col in num_cols:
        plt.figure(figsize=(6, 4))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col}')
        plt.tight_layout()
        plt.show()

    # Time-Series Analysis
    if datetime_cols:
        time_col = datetime_cols[0]
        df = df.sort_values(by=time_col)
        df.set_index(time_col, inplace=True)
        print(f"\n🕒 Time-Series Analysis using '{time_col}'")

        for col in num_cols:
            plt.figure(figsize=(10, 4))
            df[col].plot()
            plt.title(f'Time-Series Plot of {col}')
            plt.xlabel('Time')
            plt.ylabel(col)
            plt.tight_layout()
            plt.show()

            # Rolling Statistics
            plt.figure(figsize=(10, 4))
            df[col].rolling(window=12).mean().plot(label='Rolling Mean')
            df[col].rolling(window=12).std().plot(label='Rolling Std')
            plt.title(f'Rolling Statistics of {col}')
            plt.legend()
            plt.tight_layout()
            plt.show()

            # Seasonal Decomposition
            try:
                decomposition = seasonal_decompose(df[col].dropna(), model='additive', period=12)
                decomposition.plot()
                plt.suptitle(f'Seasonal Decomposition of {col}', y=1.02)
                plt.tight_layout()
                plt.show()
            except Exception as e:
                print(f"Could not perform seasonal decomposition for {col}: {e}")

    print("\n✅ EDA Completed")

# Example usage:
# df = pd.read_csv('your_dataset.csv')
# eda_template(df)